In [1]:
import pandas as pd
import json
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC


# Store dataset in a Pandas Dataframe
df_ = pd.read_csv("preprocessed_Modelisation.csv", index_col='Unnamed: 0')
pd.set_option('display.max_columns', None)

#Supprimer les colonnes non pertinentes restantes
df_=df_.drop(['state_changed_at_date','state_changed_at_weekday', 'state_changed_at_month','state_changed_at_year','launched_at_date','deadline_date'],axis=1)

#remplacer les modalités de la variable cible par 0 et 1
df_['state']=df_['state'].replace({'failed':0, 'successful':1})

#Aperçu du jeu de données
df_.head()

,backers_count,category,country,goal,staff_pick,state,usd_pledged,usd_type,deadline_weekday,deadline_month,deadline_year,launched_at_weekday,launched_at_month,launched_at_year
0,2,Comedy,US,120000,False,0,26.00,international,2,12,2022,4,12,2022
17,150,Literary Spaces,GB,3600,False,1,5028.64,international,2,11,2022,6,10,2022
18,627,Literary Spaces,US,60000,True,1,77379.85,international,4,11,2022,4,10,2022
19,365,Literary Spaces,US,50000,True,1,55804.32,international,2,11,2022,0,10,2022
20,22,Literary Spaces,US,1200,False,1,1291.00,international,0,11,2022,5,10,2022


# Modèles de classification binaire

In [2]:
#identification des variables catégorielles et numériques
cat_cols = list(df_.drop('state',axis=1).select_dtypes(['object','bool']))
num_cols = list(df_.drop('state',axis=1).select_dtypes('number'))
print("cat_cols:",cat_cols)
print("num_cols:", num_cols,"\n")

#encodage des variables catégorielles
df1 =pd.get_dummies(df_['staff_pick'], prefix ='staff_pick')
df2 =pd.get_dummies(df_['usd_type'],prefix='usd_type')
df3 =pd.get_dummies(df_['category'])
df4 =pd.get_dummies(df_['country'])

df5 = pd.concat([df_,df1,df2,df3,df4],axis=1)
df5.drop(['staff_pick','category', 'usd_type', 'country'],axis=1,inplace=True)

#Séparer les données
X = df5.drop(['state','usd_pledged','backers_count'],axis=1)
y = df5['state']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

#standardisation des variables numériques
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("Train Set:", X_train.shape)
print("Test Set:", X_test.shape)

cat_cols: ['category', 'country', 'staff_pick', 'usd_type']
num_cols: ['backers_count', 'goal', 'usd_pledged', 'deadline_weekday', 'deadline_month', 'deadline_year', 'launched_at_weekday', 'launched_at_month', 'launched_at_year'] 

Train Set: (2531, 31)
Test Set: (633, 31)


### Arbre linéaire

In [3]:
tree = DecisionTreeClassifier()

#Entraîner le modèle sur le jeu d'entraînement
tree.fit(X_train, y_train)

#Faire les prédictions sur le jeu de test avec méthode predict
y_pred_tree = tree.predict(X_test)

#Évaluer le modèle à l'aide de la méthode score
print("Score train:",tree.score(X_train, y_train))
print("Score test:",tree.score(X_test, y_test),"\n\n")

#Matrice de confusion et rapport de classification
display(pd.crosstab(y_test,y_pred_tree, rownames=['Realité'], colnames=['Prédiction']))
print("\n\n",classification_report(y_test, y_pred_tree))

Score train: 0.9996048992493086
Score test: 0.8973143759873617 




Prédiction,0,1
Realité,,
0,63,33
1,32,505




               precision    recall  f1-score   support

           0       0.66      0.66      0.66        96
           1       0.94      0.94      0.94       537

    accuracy                           0.90       633
   macro avg       0.80      0.80      0.80       633
weighted avg       0.90      0.90      0.90       633



### Régression logistique 

In [4]:
reg_log = LogisticRegression()

#Entraîner le modèle sur le jeu d'entraînement
reg_log.fit(X_train, y_train)

#Faire les prédictions sur le jeu de test avec méthode predict
y_pred_reg_log = reg_log.predict(X_test)

#Évaluer le modèle à l'aide de la méthode score
print("Score train:",reg_log.score(X_train, y_train))
print("Score test:",reg_log.score(X_test, y_test),"\n\n")

#Matrice de confusion et rapport de classification
display(pd.crosstab(y_test,y_pred_reg_log, rownames=['Realité'], colnames=['Prédiction']))
print("\n\n",classification_report(y_test, y_pred_reg_log))

Score train: 0.9126827340971948
Score test: 0.9052132701421801 




Prédiction,0,1
Realité,,
0,69,27
1,33,504




               precision    recall  f1-score   support

           0       0.68      0.72      0.70        96
           1       0.95      0.94      0.94       537

    accuracy                           0.91       633
   macro avg       0.81      0.83      0.82       633
weighted avg       0.91      0.91      0.91       633



### Random Forest 

In [5]:
rf = RandomForestClassifier()

#Entraîner le modèle sur le jeu d'entraînement
rf.fit(X_train, y_train)

#Faire les prédictions sur le jeu de test avec méthode predict
y_pred_rf = rf.predict(X_test)

#Évaluer le modèle à l'aide de la méthode score
print('Score train:', rf.score(X_train, y_train))
print('Score test:', rf.score(X_test, y_test),"\n\n")

#Matrice de confusion et rapport de classification
display(pd.crosstab(y_test,y_pred_rf, rownames=['Realité'], colnames=['Prédiction']))
print("\n\n",classification_report(y_test, y_pred_rf))

Score train: 0.9996048992493086
Score test: 0.9162717219589257 




Prédiction,0,1
Realité,,
0,73,23
1,30,507




               precision    recall  f1-score   support

           0       0.71      0.76      0.73        96
           1       0.96      0.94      0.95       537

    accuracy                           0.92       633
   macro avg       0.83      0.85      0.84       633
weighted avg       0.92      0.92      0.92       633



### SVC 

In [6]:
from sklearn.svm import SVC
svc = SVC()

#Entraîner le modèle sur le jeu d'entraînement
svc.fit(X_train, y_train)

#Faire les prédictions sur le jeu de test avec méthode predict
y_pred_svc = svc.predict(X_test)

#Évaluer le modèle à l'aide de la méthode score
print('Score train:', rf.score(X_train, y_train))
print('Score test:', rf.score(X_test, y_test),"\n\n")

#Matrice de confusion et rapport de classification
display(pd.crosstab(y_test,y_pred_svc, rownames=['Realité'], colnames=['Prédiction']))
print("\n\n",classification_report(y_test, y_pred_svc))

Score train: 0.9996048992493086
Score test: 0.9162717219589257 




Prédiction,0,1
Realité,,
0,75,21
1,32,505




               precision    recall  f1-score   support

           0       0.70      0.78      0.74        96
           1       0.96      0.94      0.95       537

    accuracy                           0.92       633
   macro avg       0.83      0.86      0.84       633
weighted avg       0.92      0.92      0.92       633



### XGBoost 

In [7]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from xgboost.compat import XGBClassifierBase
from xgboost import XGBClassifier

xgb = XGBClassifier()

#Entraîner le modèle sur le jeu d'entraînement
xgb.fit(X_train, y_train)

#Faire les prédictions sur le jeu de test avec méthode predict
y_pred_xgb =xgb.predict(X_test) 

#Évaluer le modèle à l'aide de la méthode score
print('Score train:', xgb.score(X_train, y_train))
print('Score test:', xgb.score(X_test, y_test),"\n\n")

#Matrice de confusion et rapport de classification
display(pd.crosstab(y_test,y_pred_xgb, rownames=['Realité'], colnames=['Prédiction']))
print("\n\n",classification_report(y_test, y_pred_xgb))

Score train: 0.9996048992493086
Score test: 0.9146919431279621 




Prédiction,0,1
Realité,,
0,70,26
1,28,509




               precision    recall  f1-score   support

           0       0.71      0.73      0.72        96
           1       0.95      0.95      0.95       537

    accuracy                           0.91       633
   macro avg       0.83      0.84      0.84       633
weighted avg       0.92      0.91      0.92       633



### AdaBoost 

In [1]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier ()
ada = AdaBoostClassifier(base_estimator=tree, n_estimators=400)

#Entraîner le modèle sur le jeu d'entraînement
ada.fit(X_train, y_train)

#Faire les prédictions sur le jeu de test avec méthode predict
y_pred_ada = ada.predict(X_test)

#Évaluer le modèle à l'aide de la méthode score
print('Score train:', ada.score(X_train, y_train))
print('Score test:', ada.score(X_test, y_test),"\n\n")

#Matrice de confusion et rapport de classification
display(pd.crosstab(y_test,y_pred_ada, rownames=['Realité'], colnames=['Prédiction']))
print("\n\n",classification_report(y_test, y_pred_ada))

NameError: name 'tree' is not defined

# Comparaison de la performance des modèles testés et sélection du/des 2 meilleur(s)¶

Selon l'accuracy score, nous obtenons 2 modèles dont les performances sont quasi identiques: SVC et RandomForest. En effet leur score sur le jeu de test d'environ 0.88 . Cela signifique qu'ils ont tous les 2 la proportion de prédictions correctes les plus élevées de tous les modèles testés.

Ceci dit, le RandomForest a le f1-score pour la classe 1 le plus élevé (0.92) mais le SVC le suit de près avec un f1-score de 0.91.

Nous choisissons le modèle le plus performant comme étant le RandomForest même si le SVC l'est tout autant.

# Optimisation du/des modèles sélectionnés 

### Avec GridSearchCV 

In [ ]:
#Optimisation de RandomForest en utilisant GridSearchCV

%%time 
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators':[100,150,300],
    'criterion':['gini', 'entropy', 'log_loss'],
    'max_depth':[5,10,15],
}

# Initialiser RandomForest
rf_clf = RandomForestClassifier()

# Initialiser GridSearchCV
grid = GridSearchCV(rf_clf, param_grid, cv=5, scoring='accuracy')

# Entraîner le modèle
grid.fit(X_train, y_train)

In [ ]:
#Afficher les paramètres optimaux du modèle RandomForest selon notre GridSearchCV
print("Les meilleurs paramètres de RandomForest sont:",grid.best_params_)

### Avec la méthode de réduction des dimensions 

In [ ]:
#Appliquer la méthode de réduction des dimensions sur le modèle RandomForest
#pour déterminer si la performance du modèle peut être améliorée de cette façon

from sklearn.decomposition import PCA

pca = PCA(n_components=10)

rf_reduit = RandomForestClassifier()

#Appliquer la technique de réduction des dimensions au jeu d'entraînement et de test
X_train_reduit = pca.fit_transform(X_train)
X_test_reduit = pca.transform(X_test)

#Entraîner les dimensions réduites sur le modèle le plus performant identifié plus haut
rf_reduit.fit(X_train_reduit, y_train)

#Faire les prédictions sur le modèle réduit
y_pred_reduit = rf_reduit.predict(X_test_reduit)

#Évaluer le modèle à l'aide de la méthode score
print('Score train:', rf_reduit.score(X_train_reduit, y_train))
print('Score test:', rf_reduit.score(X_test_reduit, y_test),"\n\n")

#Matrice de confusion et rapport de classification
display(pd.crosstab(y_test,y_pred_reduit, rownames=['Realité'], colnames=['Prédiction']))
print("\n\n",classification_report(y_test, y_pred_reduit))

##Conclusion : en utilisant l'ACP sur RandomForest, le score test est très légèrement 
#moins bon sur le jeu réduit que sur le jeu non réduit : 0.86 vs 0.88

##ce ne sera finalement pas vraiment utile pour notre analyse. De plus, notre jeu de données est sans 
#doute trop petit pour que cette méthode soit utile.

# Interprétabilité du/des modèle(s)

In [ ]:
#Afficher l'importance de chaque colonne pour le modèle RandomForest rf

rf.feature_importances_


In [ ]:
#Dataframe contenant les variables avec leurs importances

columns=['goal', 'deadline_weekday', 'deadline_month', 'deadline_year',
       'launched_at_weekday', 'launched_at_month', 'launched_at_year',
       'staff_pick_False', 'staff_pick_True', 'usd_type_domestic',
       'usd_type_international', 'Audio', 'Comedy', 'Cookbooks', 'Design',
       'Graphic Novels', 'Literary Journals', 'Literary Spaces', 'Photo',
       'Plays', 'R&B', 'Sound', 'Spaces', 'Toys', 'Webcomics', 'Asia', 'CA',
       'Europe', 'GB', 'MX', 'Oceania', 'US']

X_features=list(rf.feature_importances_)

df_feature=pd.DataFrame({'Feature':columns,
                         'Score':X_features})

df_feature=df_feature.sort_values(['Score'], ascending=[False])

df_feature

In [ ]:
#Additionner les importances des sous-colonnes pour les regrouper en 6 variables

L=[]
imp_deadline = 0
imp_launched = 0
imp_staff = 0
imp_usdt = 0
imp_categ = 0
imp_geo = 0 

if i in range(1,31):
  for i in [1,2,3]:
    imp_deadline += rf.feature_importances_[i]
  for i in [4,5,6]:
    imp_launched += rf.feature_importances_[i]
  for i in [7,8]:
    imp_staff += rf.feature_importances_[i]
  for i in [9,10]:
    imp_usdt += rf.feature_importances_[i]
  for i in range(11, 25):
    imp_categ += rf.feature_importances_[i]
  for i in range(25, 31):
    imp_geo += rf.feature_importances_[i]

L.append([imp_deadline, imp_launched, imp_staff,imp_usdt,imp_categ,imp_geo])


importances = [rf.feature_importances_[0],imp_deadline, imp_launched, imp_staff,imp_usdt,imp_categ,imp_geo]
variables = ['goal', 'deadline', 'launched_at', 'staff', 'usd_type', 'category', 'zone_geo']

df_importances = pd.DataFrame({'Variable':variables, 'Importance':importances}).sort_values(['Importance'], ascending=[False])
df_importances

In [ ]:
#Faire un barplot de la somme des importances des variables

import matplotlib.pyplot as plt

plt.figure(figsize=(20,10))
fig, ax = plt.subplots()
ax= plt.bar(variables,importances, width=0.7)
plt.xticks(rotation=45)
plt.title('Importances des variables',fontsize=14)
plt.xlabel('Variables',fontsize=12, labelpad=10)
plt.ylabel('Importance',fontsize=12, labelpad=10)
plt.show();

In [ ]:
#comparer y_test (données réélles) aux données prédites par le modèle RandomForest (y_pred_rf)
predictions = pd.DataFrame({'y_test': y_test, 'y_pred_rf': y_pred_rf})
predictions.head()

In [ ]:
#Déterminer sur quelles campagnes le modèle RandomForest a fait des erreurs

indices_erreur = y_pred_rf != y_test
X_test_erreur = X_test[indices_erreur]
erreurs = pd.DataFrame(X_test_erreur['goal']).T

X_test_erreur

In [ ]:
#étudier la répartition des valeurs de "goal" pour identifier si cette variable peut être la cause des erreurs
X_test_erreur['goal'].describe()

#en regardant la valeur de "goal" pour chaque ligne surlaquelle le modèle s'est trompé,
#on remarque que la valeur est soit très faible soit très élevée: l'écart type est très
#grand ce qui signifie une grande variabilité dans les données. De plus la médiane
#est très largement inférieure à la moyenne ce qui peut suggérer des valeurs extrêmes.
#on peut en conclure que les erreurs sont faites sur les valeurs extrêmes de "goal"